# Лабораторная работа 4

In [1]:
!pip install pykeen

### Импорт зависимостей

In [2]:
import pykeen
import torch
from pykeen.datasets import FB15k
from pykeen.pipeline import pipeline
from pykeen.predict import predict_triples

### Загрузка датасета

In [3]:
dataset = FB15k()

In [4]:
print(dataset)

FB15k(training_path="/home/maxim/.data/pykeen/datasets/fb15k/FB15k/freebase_mtr100_mte100-train.txt", testing_path="/home/maxim/.data/pykeen/datasets/fb15k/FB15k/freebase_mtr100_mte100-test.txt", validation_path="/home/maxim/.data/pykeen/datasets/fb15k/FB15k/freebase_mtr100_mte100-valid.txt")


In [5]:
training_data = dataset.training.mapped_triples
print(training_data[:5])

tensor([[    0,   785, 14076],
        [    0,   795, 14076],
        [    0,   796,     0],
        [    0,   798, 13352],
        [    0,   800, 13352]])


### Инициализация и обучение модели

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [7]:
pipeline_results = pipeline(
    model="TransE",      # Выбираем модель (TransE - одна из моделей для предсказания отношений)
    dataset=dataset,     # Выбираем графовый датасет (Nations)
    training_kwargs=dict(num_epochs=20),   # Количество эпох обучения
    device=device
)

No random seed is specified. Setting to 3061047092.


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1888 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/1888 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/1888 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/1888 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/1888 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=256.


Evaluating on cuda:0:   0%|          | 0.00/59.1k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 19.62s seconds


In [8]:
print(pipeline_results)

PipelineResult(random_seed=3061047092, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(14951, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(1345, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=14951, num_relations=1345, create_inverse_triples=False, num_triples=483142, path="/home/maxim/.data/pykeen/datasets/fb15k/FB15k/freebase_mtr100_mte100-train.txt"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7fb7c637d090>, losses=[0.8243102831608158, 0.40470705544418195, 0.18256092450770914, 0.1049979444577406, 0.07771726055425923, 0.06506027645920798, 0.05861009783762649, 0.054501131912930156, 0.05019029578410081, 0.04782042517504355, 0.04662295171134648, 0.04452158704447418, 0.043862366370099076, 0.042037775855

### Triple Scoring

In [9]:
pack = predict_triples(model=pipeline_results.model, triples=dataset.validation)
df = pack.process(factory=pipeline_results.training).df

In [10]:
df.nlargest(n=100, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
47201,13758,/m/0l15bq,937,/music/performance_role/track_performances./mu...,2879,/m/01w4dy,-2.951971
19086,5792,/m/0342h,937,/music/performance_role/track_performances./mu...,952,/m/018j2,-3.087107
38173,11244,/m/0bjkk9,22,/american_football/football_team/current_roste...,4439,/m/02g_7z,-3.211626
46438,13544,/m/0k1jg,795,/location/hud_county_place/county,13544,/m/0k1jg,-3.242777
9041,2893,/m/01w724,90,/award/award_nominee/award_nominations./award/...,937,/m/018dyl,-3.310454
...,...,...,...,...,...,...,...
5617,1739,/m/01hkhq,91,/award/award_nominee/award_nominations./award/...,74,/m/011ywj,-3.828223
40575,11881,/m/0d0x8,803,/location/location/adjoin_s./location/adjoinin...,9599,/m/06yxd,-3.830260
6702,2107,/m/01m7pwq,95,/award/award_winner/awards_won./award/award_ho...,4318,/m/02dbp7,-3.830392
23342,7154,/m/042v_gx,937,/music/performance_role/track_performances./mu...,6341,/m/03gvt,-3.830773
